# Домашнее задание 7. Градиентный бустинг над решающими деревьями. Регрессия. 

Дедлайн: 07.06.2020 23:59

При градиентном бустинге деревья обучаются итеративно. Каждое новое дерево обучается на ошибках предыдущего. 
Алгоритм обучения градиентного бустинга:
+ $T$ - количество деревьев
+ $\gamma$ - размер шага (learning_rate)
+ $\{ (x_i, y_i )\}_{i=1}^N$ - обучающая выборка   


1. Инициализировать массив предсказаний $prediction$ ансамбля, заполнив его нулями. 
2. For $t$ in $1...T$      
  2.1. Посчитать остатки - антиградиент функции ошибки.    
  ***Ошибка:*** $mse(y,prediction) = (y - prediction)^2$    
  ***Градиент:*** $\nabla_{prediction} mse(y,prediction) = prediction - y$   
  ***Антиградиент:*** $residuals_t = y - prediction$    
  2.2. Обучить дерево $b_t$, обучающая выборка:$ \{ (x_i, residuals_{t, i} ) \}_{i=1}^N$ 
  (в качестве целевой переменной выступают остатки)   
  2.3. Сделать предсказание обученным деревом:     
    $prediction_t = b_t(x)$    
  2.4. Прибавить предсказанние текущей модели умноженное на размер шага к вектору предсказаний ансамбля: $prediction \mathrel{+}= \gamma*prediction_t$    




Итоговый ансамбль имеет вид:  $a(x) = \sum_{t = 1}^T (\gamma b_t(x))$

       

Ваша задача - заполнить пропуски # YOUR CODE HERE, и выполнить код

In [0]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

Обязательно фиксируем random state. 

In [0]:
SEED = 22
np.random.seed(SEED)

Реализуем простой градиентный бустинг, в качестве базового алгоритма используем DecisionTreeRegressor из sklearn. 

In [0]:
class SimpleGBRegressor:

    def __init__(self, n_estimators=10, max_depth=5, 
                 min_samples_leaf=1, learning_rate=0.1, 
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.learning_rate = learning_rate
        self.random_state = random_state
        self.trees = []
        self.was_fit = False

    def fit(self, X, y):
        # убедиться что в X и y одинаковое число элементов
        assert X.shape[0] == y.shape[0]
        # инициализировать массив с предсказаниями, заполнив нулями
        prediction = np.zeros(X.shape[0])

        # обучаем деревья
        for i in range(self.n_estimators):
            # посчитать остатки
            residual = y - prediction

            # инициализировать дерево с нужными параметрами
            tree = DecisionTreeRegressor(max_depth = self.max_depth,
                                         min_samples_leaf = self.min_samples_leaf,
                                         random_state = self.random_state)

            # обучить дерево
            tree.fit(X,residual)

            # сделать предсказание текущего дерева
            tree_prediction = tree.predict(X)

            # сохранить обученное дерево
            self.trees.append(tree)

            # обновить вектор предсказаний модели
            prediction += self.learning_rate*tree_prediction

        self.was_fit = True
        return self

    def predict(self, X):
        # если модель не была обучена, печатаем сообщение об этом и вовращаем None
        if self.was_fit == False:
          print('Модель не была обучена')
          return None

        # инициализировать массив с предсказаниями
        y_pred = np.zeros(X.shape[0])
        
        # добавить прогнозы деревьев
        for tree in self.trees:
            tree_prediction = tree.predict(X)
            y_pred += self.learning_rate*tree_prediction


        # убедиться что в X и y одинаковое число элементов
        assert X.shape[0] == y_pred.shape[0]
        return y_pred
        

## Провеярем нашу модель


Загружаем датасет Diabetes. 

In [0]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

In [8]:
print(diabetes['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bra

In [0]:
data = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
target = diabetes.target

In [10]:
data.head(10)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
5,-0.092695,-0.044642,-0.040696,-0.019442,-0.068991,-0.079288,0.041277,-0.076395,-0.041180,-0.096346
6,-0.045472,0.050680,-0.047163,-0.015999,-0.040096,-0.024800,0.000779,-0.039493,-0.062913,-0.038357
7,0.063504,0.050680,-0.001895,0.066630,0.090620,0.108914,0.022869,0.017703,-0.035817,0.003064
8,0.041708,0.050680,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014956,0.011349
9,-0.070900,-0.044642,0.039062,-0.033214,-0.012577,-0.034508,-0.024993,-0.002592,0.067736,-0.013504


Делим на обучающую и тестовую выборку, размер тестовой выборки - 30%. Не забываем про random state.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=SEED)

Обучаем нашу модель и делаем предсказание по тестовой выборке. 
Параметры модели:
+ n_estimators = 100
+ random_state - зафиксированный нами ранее
+ остальные значения по умолчанию 

In [38]:
# создаем и обучаем модель
my_model = SimpleGBRegressor(n_estimators=100, random_state=SEED)
my_model.fit(X_train, y_train)

In [0]:
# делаем предсказание по тестовой выборке
y_pred = my_model.predict(X_test)

Считаем MSE

In [41]:
# оцениваем качество
mse = np.mean(np.square(y_test - y_pred))
print(mse)

3749.674167308474


Сравниваем с реализацией аналогичного алгоритма из sklearn c **такими же** параметрами (будьте внимательны)

In [43]:
# создаем и обучаем модель
sklearn_model = GradientBoostingRegressor(n_estimators=100,
                                          max_depth=5, 
                                          random_state=SEED)
sklearn_model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=22, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
# делаем предсказание по тестовой выборке
y_pred_sklearn = sklearn_model.predict(X_test)

In [45]:
# оцениваем качество
mse = np.mean(np.square(y_test - y_pred_sklearn))
print(mse)

3725.914043073783
